# Import and Helper functions

In [1]:
# -- fix path --
from pathlib import Path
import sys
sys.path.append(str(Path('..').resolve()))
from source.resources import *
from source.metrics import *
from source.helper import *
from source.preprocessor import *
from source.constants import *
import Levenshtein
import wordfreq

from nltk import word_tokenize
from functools import lru_cache
import unicodedata
from collections import Counter
# from nltk import word_tokenize
import nltk
import magic
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
# from spacy.cli.download import download as spacy_download
# nlp = spacy.load("en_core_web_sm")

# from joblib import Memory
current_dir = Path('.')

DATASETS = [Dataset.BenchLS, Dataset.LexMTurk, Dataset.NNSeval, Dataset.SemEval2012, Dataset.TSAR_EN]


[nltk_data] Downloading package stopwords to /Users/kim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_spacy(lang='en'):
    models = {'en': 'en_core_web_sm', 'es': 'es_core_news_sm', 'pt':'pt_core_news_sm', 'fr':'fr_core_news_sm'} #efficiency
    # models = {'en': 'en_core_web_trf', 'es': 'es_dep_news_trf', 'pt':'pt_core_news_lg', 'fr':'fr_dep_news_trf'} #accuracy
    spacy.prefer_gpu()
    try:
        # If not present, download 'en_core_web_sm'
        return spacy.load(models[lang]) 
    except OSError:
        from spacy.cli.download import download as spacy_download

        spacy_download(models[lang])
        return spacy.load(models[lang])

# Spanish

In [3]:
data = load_dataset(Dataset.TSAR_ES)
data

,text,complex_word,complex_word_index,candidates
0,"), fue un pianista y compositor folclórico, y ...",folclórico,7,"[[""tradicional""], [""pintoresco"", ""t\u00edpico""..."
1,"a comienzos de la década de 1980, se trasladó ...",labrarse,20,"[[""formarse""], [""construirse"", ""hacerse""], [""g..."
2,a lo largo de sus más de veinte años de experi...,tertulias,33,"[[""charlas""], [""reuniones""], [""debates""], [""co..."
3,a pesar de las pocas bajas (menos de 500 en to...,propiciado,38,"[[""provocado""], [""favorecido"", ""desencadenado""..."
4,a pesar de las pocas bajas (menos de 500 en to...,decadencia,40,"[[""ca\u00edda""], [""debilitaci\u00f3n""], [""caid..."
...,...,...,...,...
376,una bomba peristáltica es un tipo de bomba hid...,hidráulica,8,"[[""hidr\u00e1ulica""], [""hidromec\u00e1nica""], ..."
377,"una noche,un extraño personaje llamado dedo po...",secuaces,53,"[[""seguidores""], [""c\u00f3mplices""], [""compa\u..."
378,"utilizó el seudónimo ""etpa"" (ermelinda talea p...",seudónimo,2,"[[""alias""], [""apodo"", ""sobrenombre""], [""nombre..."
379,"villasequilla se llamó antes ""villaseca de val...",dependencia,18,"[[""vinculaci\u00f3n""], [""subordinaci\u00f3n""],..."


In [4]:
pos_types = {}
pos_types_words = {}
nlp = load_spacy('es')
for i in tqdm(range(len(data))):
    row = data.iloc[i]
    text = row['text']
    # print(text)
    complex_word = row['complex_word']
    doc = nlp(text)
    for token in doc:
        # print(token.text, token.pos_)
        if token.text == complex_word:
            pos = token.pos_
            # print(token.text, pos)
            if pos in pos_types:
                pos_types[pos] += 1 
            else:
                pos_types[pos] = 1
            if pos in pos_types_words:
                pos_types_words[pos].append(token.text)
            else: 
                pos_types_words[pos] = [token.text]
    # break 

print(pos_types)


100%|██████████| 381/381 [00:01<00:00, 314.00it/s]

{'ADJ': 77, 'VERB': 72, 'NOUN': 225, 'PROPN': 4, 'ADV': 4}


In [5]:
for key, values in pos_types_words.items():
    print(key, ', '.join(values))
    print('='*80)


ADJ folclórico, difunto, célebre, extinta, indispuesto, historiográfica, burlesca, homónimo, sobrenatural, mortífero, pictóricas, telúricos, monástico, empotrado, acreditada, ancestral, metafórica, épico, ovina, desacreditados, topónimo, feudos, robusto, estirpe, emblemáticos, anglosajón, divulgativo, interino, evangélicos, célebre, prominentes, adyacente, misericordioso, sarcástica, protagónicas, enchufados, reputada, sublevadas, versátiles, representativo, emblemáticos, inhabitual, encuadrada, datado, contestataria, malcriado, primordialmente, aleatorias, burguesas-conservadoras, burocráticas, larvas, arraigada, imputado, prolífico, irascible, profanos, incierta, calibrados, minimalista, rudimentarios, aristocrática, dotado, abruptas, sobrehumanas, vertiente, devastador, lírico, sincrónica, colmatado, reducidísimo, condecorado, homónimo, innato, desafortunado, hidráulica, secuaces, escéptica
VERB labrarse, propiciado, albergaba, mimetizar, cuchicheaban, debutó, propiciara, asumiendo,

# English

In [6]:
data = load_dataset(Dataset.TSAR_EN)
data

,text,complex_word,complex_word_index,candidates
0,this discovery helped to establish yet another...,prototype,28,"[[""model"", ""sample""], [""prototype""], [""origina..."
1,putin was expected to formally register later ...,authoritarian,29,"[[""dictatorial""], [""controlling"", ""authoritari..."
2,"#7-8 despite the fog, other flights are report...",collision,18,"[[""crash""], [""impact"", ""accident""], [""collisio..."
3,"the larva grows to about 120-130 mm , and pupa...",pupates,9,"[[""grows""], [""pupates""], [""develops""], [""matur..."
4,gorbachev told the interfax news agency that a...,discontent,20,"[[""dissatisfaction""], [""unhappiness""], [""unres..."
...,...,...,...,...
381,"pearl s. buck died of lung cancer on march 6 ,...",interred,20,"[[""buried""], [""burried""], [""entombed"", ""inhume..."
382,his satirical street art and subversive epigra...,epigrams,6,"[[""quips"", ""jokes""], [""gems"", ""puns""], [""jest""..."
383,brevard county was the scene of six homicides ...,homicides,7,"[[""murders""], [""killings""], [""assassinations""]..."
384,#5-4 northern mali was seized by islamic milit...,seized,5,"[[""captured"", ""grabbed""], [""taken""], [""overtak..."


In [7]:
pos_types = {}
pos_types_words = {}
nlp = load_spacy('en')
for i in tqdm(range(len(data))):
    row = data.iloc[i]
    text = row['text']
    # print(text)
    complex_word = row['complex_word']
    doc = nlp(text)
    for token in doc:
        # print(token.text, token.pos_)
        if token.text == complex_word:
            pos = token.pos_
            # print(token.text, pos)
            if pos in pos_types:
                pos_types[pos] += 1 
            else:
                pos_types[pos] = 1
            if pos in pos_types_words:
                pos_types_words[pos].append(token.text)
            else: 
                pos_types_words[pos] = [token.text]
    # break 

print(pos_types)


100%|██████████| 386/386 [00:01<00:00, 330.60it/s]

{'NOUN': 177, 'ADJ': 60, 'VERB': 139, 'ADV': 12, 'PROPN': 2}


In [8]:
for key, values in pos_types_words.items():
    print(key, ', '.join(values))
    print('='*80)


NOUN prototype, collision, pupates, discontent, descent, offshoot, decree, deficit, canopy, condolences, scrutiny, regime, snipers, precautions, accolade, uprising, offender, encodes, residences, ringleaders, compound, exclusion, observers, toxins, authorities, monsters, detainees, scrutiny, replica, probe, aviator, repression, vulnerability, equipment, existence, proportion, motive, determination, bloodshed, credibility, exclusion, anonymity, indignation, troublemakers, abdomen, pursuit, inclusion, bombardment, colonies, segregation, residence, regulators, regulators, shrapnel, bureau, anonymity, enquiries, incident, incident, offender, anonymity, clemency, sanctuary, demonstrations, allege, bloodshed, adversary, renderings, nobility, colonies, militiamen, carnage, stint, atrocities, maniacs, candidacy, auctioneers, bloodshed, organisers, clearance, sympathy, composition, sanctions, flashpoint, assumption, casualties, acquisition, pedestal, regime, regime, demonstrations, sentiment, s

# Portuguese

In [9]:
data = load_dataset(Dataset.TSAR_PT)
data

,text,complex_word,complex_word_index,candidates
0,ao prever a distribuição gratuita de uniforme ...,controvérsia,20,"[[""pol\u00eamica""], [""contradi\u00e7\u00e3o""],..."
1,a matéria ainda precisa passar pela câmara dos...,matéria,1,"[[""assunto"", ""lei""], [""proposta"", ""conte\u00fa..."
2,o acessório polêmico entrou no projeto de auto...,polêmico,2,"[[""controverso""], [""discut\u00edvel""], [""duvid..."
3,toda a operação teria sido armada para vingar ...,armada,5,"[[""criada""], [""planejada""], [""organizada"", ""pr..."
4,blitze barreiras e operações em profusão foram...,profusão,5,"[[""abund\u00e2ncia""], [""fartura"", ""quantidade""..."
...,...,...,...,...
381,para provar a idéia os pesquisadores compilara...,compilaram,6,"[[""reuniram""], [""juntaram""], [""selecionaram""],..."
382,o estudo estará numa edição futura do periódic...,científico,8,"[[""tecnol\u00f3gico"", ""t\u00e9cnico""], [""mestr..."
383,cremos que elas seriam mais eficientes que os ...,inibir,11,"[[""impedir""], [""diminuir""], [""impossibilitar""]..."
384,a ficção científica americana na forma de livr...,fascinada,12,"[[""encantada"", ""apaixonada""], [""deslumbrado""],..."


In [10]:
pos_types = {}
pos_types_words = {}
nlp = load_spacy('pt')
for i in tqdm(range(len(data))):
    row = data.iloc[i]
    text = row['text']
    # print(text)
    complex_word = row['complex_word']
    doc = nlp(text)
    for token in doc:
        # print(token.text, token.pos_)
        if token.text == complex_word:
            pos = token.pos_
            # print(token.text, pos)
            if pos in pos_types:
                pos_types[pos] += 1 
            else:
                pos_types[pos] = 1
            if pos in pos_types_words:
                pos_types_words[pos].append(token.text)
            else: 
                pos_types_words[pos] = [token.text]
    # break 

print(pos_types)


100%|██████████| 386/386 [00:00<00:00, 439.37it/s]

{'NOUN': 197, 'ADJ': 61, 'VERB': 120, 'ADP': 1, 'ADV': 9}


In [11]:
for key, values in pos_types_words.items():
    print(key, ', '.join(values))
    print('='*80)


NOUN controvérsia, matéria, profusão, incremento, contenção, ocupantes, espectadores, desgaste, depredação, celebração, delegações, delegações, pontífice, rentabilidade, postulante, melancolia, ferroadas, reivindicações, prudência, divergência, conduta, premier, confinamento, fiscalização, pleitos, questão, grupo, funcionários, abrasador, conjecturas, aparelho, legenda, luminosidade, resolução, matriarca, quitutes, reflexão, rigor, mortandade, escalões, delegação, escalões, emissário, restrito, restrição, transferência, formatação, procedimentos, fundação, degradação, adesão, patrulha, eqüinos, pluralidade, itens, sanções, mordomias, mordomias, correção, turbulência, correções, malentendido, apenados, subordinação, colisão, notificação, assimetria, concretização, restrições, abastecimento, chancela, abastecimento, controvérsias, armazenamento, determinação, apenados, panificadora, panificadora, ampliação, descrença, proprietária, praga, expedição, detritos, segmento, precaução, impacto